In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

from langchain_openai import OpenAIEmbeddings, OpenAI
from langchain_core.prompts import PromptTemplate

model_name = os.getenv("LLM_MODEL") or "gpt-4o-mini"
model_provider = os.getenv("LLM_PROVIDER") or "openai"

import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings, ChatOpenAI


from langchain import hub



USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
#### INDEXING ####
loader = WebBaseLoader(
    web_paths=["https://news.naver.com/section/101"],
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("sa_text", "sa_item SECTION_HEADLINE")
        )
    )
)

docs = loader.load()

In [3]:
# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",
    chunk_size=300,
    chunk_overlap=50
)
splits = text_splitter.split_documents(docs)


In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(
        model="text-embedding-3-large",   
    )
)

In [5]:
chroma_retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs=dict(
        k=1,
        fetch_k=4
    )
)



In [6]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
bm25_retriever = BM25Retriever.from_documents(splits)
bm25_retriever.k = 2

ensemble_retriever = EnsembleRetriever(
    retrievers=[chroma_retriever, bm25_retriever],
    weights=[0.2, 0.8]
)


In [7]:
query = "향후 집갑에 대해서 알려줘"

In [8]:
def print_docs(docs):
    print("="*100)    
    for doc in docs:
        print(doc.page_content)
        print("-"*100)

In [10]:
docs = ensemble_retriever.invoke(query)

In [11]:
print(docs)

[Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='美 연준 "금리 인하 어려울 것"…비트코인 11.1만 달러 하락[코인브리핑]\n\n최재헌 기자 연준 매파적 발언에 위험자산 투자심리 위축 미국 연방준비제도(연준)가 다음 달 기준금리 인하 가능성을 낮게 점치자, 비트코인(BTC) 가격이 11만 1000달러 선까지 밀려났다. 22일 오전 9시 40분\n\n\n뉴스1\n\n48분전'), Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content="[단독]프랑스 가스 제조사 에어리퀴드, '몸값 4조' DIG에어가스 본계약 체결\n\n이 기사는 08월 22일 11:42 마켓인사이트에 게재된 기사입니다. 글로벌 산업용 가스 제조사 에어리퀴드가 DIG에어가스를 인수한다. 에어리퀴드는 이번 인수합병(M&A)으로 2014년 대성산업가스 지분을 전량 매각\n\n\n한국경제\n\n14분전\n\n\n\n\n\n\n\n\n그룹 총수들 총출동...트럼프에 선물 보따리 안긴다"), Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content="헤럴드경제\n\n32분전\n\n\n\n\n\n\n\n\nLS일렉트릭, 美 고율관세 피하는 법…변압기 대신 배전반\n\nLS일렉트릭이 미국의 철강·알루미늄 파생제품에 최대 50%의 고율관세가 부과되는 상황에서 '관세 무풍지대'로 꼽히는 배전반 전략을 강화한다. 경쟁사가 관세 부담이 큰 변압기의 현지 생산 확대에 나서는 것과 달리, L\n\n\n아시아경제\n\n33분전\n\n\n\n\n\n\n\n\n외국인이 전액 예금으로?…실거주 안 하면 막힌다")]


In [12]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

template = """
당신은 AI 언어 모델 조수입니다. 당신의 임무는 주어진 사용자 질문에 대해 벡터 데이터베이스에서 관련 문서를 검색할 수 있도록 다섯 가지 다른 버전을 생성하는 것이빈다.
사용자 질문에 대한 여러 관점을 생성함으로써, 거리 기반 유사성 검색의 한계를 극복하는 데 도움을 주는 것이 목표입니다.
각 질문은 새로운 줄로 구분하여 제공하세요. 원본 질문: {question}
"""

prompt_perspectives = ChatPromptTemplate.from_template(template)


generate_queries = (
    prompt_perspectives
    | ChatOpenAI(model=model_name, temperature=0)
    | StrOutputParser()
    |(lambda x: x.split("\n"))
)

generated_query = generate_queries.invoke("집값의 향방?")
print(generated_query)



['집값이 앞으로 어떻게 변할까요?  ', '향후 부동산 가격은 어떤 방향으로 움직일까요?  ', '주택 시장의 가격 추세는 어떻게 될지 궁금합니다.  ', '앞으로 집값 상승 또는 하락 전망은 어떠한가요?  ', '부동산 가격 변동에 영향을 미치는 요인은 무엇일까요?']


In [13]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60, top_n=2):
    """
    여러 개의 순위가 매겨진 문서 리스트를 받아, RRF(Reciprocal Rank Fusion) 공식을 사용하여 문서의 최종 순위를 계산하는 함수입니다. 
    k는 RRF 공식에서 사용되는 선택적 파라미터이며, top_n은 반환할 우선순위가 높은 문서의 개수입니다.
    """

    fused_scores = {}

    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] = 1 / (rank + 1)

    fused_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    return fused_results[:top_n]

In [14]:
question = "향후 집값에 대해서 알려줘"


In [15]:
retrievel_chain_rag_fusion = generate_queries | ensemble_retriever.map() | reciprocal_rank_fusion

In [17]:
docs = retrievel_chain_rag_fusion.invoke(query)
#print_docs(docs)



In [18]:
docs

[(Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='美 연준 "금리 인하 어려울 것"…비트코인 11.1만 달러 하락[코인브리핑]\n\n최재헌 기자 연준 매파적 발언에 위험자산 투자심리 위축 미국 연방준비제도(연준)가 다음 달 기준금리 인하 가능성을 낮게 점치자, 비트코인(BTC) 가격이 11만 1000달러 선까지 밀려났다. 22일 오전 9시 40분\n\n\n뉴스1\n\n48분전'),
  1.0),
 (Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='44분전\n\n\n\n\n\n\n\n\n아산·광명·검단, 보증수표 된 검증 입지 후속 분양… 하반기 시장 주도\n\n부동산 시장에서 옥석 가리기가 심화되면서 이미 검증된 입지에서 후속으로 공급되는 단지들이 강세를 보이고 있다. 충남 아산 탕정지구, 경기 광명뉴타운, 동탄2신도시, 충북 청주테크노폴리스 등은 최근 분양 성적을 통해 \n\n\n동아일보\n\n45분전\n\n\n\n\n\n\n\n\n[속보]법원, 티몬 회생절차 종결 결정…법정관리 졸업'),
  1.0)]

In [19]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

template = """다음 맥락을 바탕으로 질문에 답변하세요:

{context}

질문: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {
        "context": retrievel_chain_rag_fusion,
        "question": RunnablePassthrough()
    }
    | prompt
    | ChatOpenAI(model=model_name, temperature=0)
    | StrOutputParser()
)

rag_docs = final_rag_chain.invoke(question)
print(rag_docs)

주어진 뉴스 맥락을 바탕으로 향후 집값 전망에 대해 말씀드리면, 현재 부동산 시장에서는 입지가 이미 검증된 지역에서 후속 분양되는 단지들이 강세를 보이고 있습니다. 즉, 충남 아산 탕정지구, 경기 광명뉴타운, 동탄2신도시, 충북 청주테크노폴리스 등 검증된 입지의 신규 분양 단지들이 좋은 분양 성적을 기록하고 있어, 이런 지역을 중심으로 집값이 상대적으로 안정적이거나 상승세를 이어갈 가능성이 높습니다.

반면, 입지가 불확실하거나 검증되지 않은 지역은 옥석 가리기가 심화되고 있어 집값 변동성이 클 수 있으니 주의가 필요합니다.

요약하면, 향후 집값은 입지와 개발 호재가 명확한 지역을 중심으로 강세를 보일 것으로 예상되며, 검증된 입지의 후속 분양 단지들이 시장을 주도할 전망입니다.
